# V5 Deep Explainer (개별 데이터 분석)

In [52]:
import shap
import torch
import torch.nn as nn
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image
import glob
import os

In [53]:
# 오분류 이미지 경로
# misclassified_path = r"C:\Users\USER\Desktop\my_git\safebaby-xai\resnet50_explain\resnet50_sample_data\V5\back"
# image_paths = glob.glob(os.path.join(misclassified_path, "*.png"))
# 단일 분석 예정

# 모델 로드
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.resnet50(weights=None) # 가중치 없는 모델
model_path = r"resnet50_sample_data\V5\best_model_V5.pth"

# Deep Explainer 사용을 위해 모든 ReLU를 inplace=False로 변경
for module in model.modules():
    if isinstance(module, nn.ReLU):
        module.inplace = False

# fc 확인 (3개 맞는지)
num_ftrs = model.fc.in_features
if model.fc.out_features != 3:
    model.fc = torch.nn.Linear(num_ftrs, 3) # 3아니면 출력 뉴런 3개로 변경

# V5 모델 가중치 로드
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval() # 모델을 평가 모드로 설정

# 기존 ResNet50 모델과 동일 전처리리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 클래스 정의
classes = ["Back", "Front", "Side"]

# 이미지 변환(ResNet50 입력 형식)
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0) # 배치 차원을 추가
    return image_tensor.to(device)

# 모델 예측 함수
def predict_class(img_tensor):
    model.eval()
    with torch.no_grad():
        logits = model(img_tensor) # 모델 예측값 (logits)
        probs = F.softmax(logits, dim=1)
        pred_class = torch.argmax(probs, dim=1).item() # 가장 높은 확률의 클래스 선택
    return classes[pred_class], probs.cpu().numpy()

# 모델 예측 함수
def model_forward(x):
    # SHAP explainer에 입력할 모델 예측 함수
    x_tensor = torch.tensor(x).permute(0, 3, 1, 2).to(device) # (batch, H, W, C) → (batch, C, H, W)
    with torch.no_grad():
        logits = model(x_tensor) # 모델 예측 수행 (logits 값 출력)
        probabilities = F.softmax(logits, dim=1) # 확률 값으로 변환
    return probabilities.cpu().numpy() # numpy 배열로 변환

### Deep Explainer 설정

In [54]:
# 랜덤 샘플 사용
background_samples = torch.randn((10, 3, 224, 224)).to(device)
explainer = shap.DeepExplainer(model, background_samples)

# 분석 및 시각화 함수
def analyze_single_image(image_path):
    img_tensor = preprocess_image(image_path)
    img_array = img_tensor.cpu().numpy().transpose(0, 2, 3, 1)[0] # (1, 3, 224, 224) → (224, 224, 3)

    # 모델 예측 수행
    pred_class_name, pred_probs = predict_class(img_tensor)
    print(f" 예측 클래스: {pred_class_name} (확률: {np.max(pred_probs) * 100:.2f}%)")

    # SHAP 실행 (한 장씩)
    shap_values = explainer.shap_values(img_tensor)
    shap_values = np.array(shap_values)[0] # (1, 3, 224, 224) → (3, 224, 224)

    # SHAP 시각화
    visualize_shap(img_array, shap_values, pred_class_name)


def visualize_shap(image, shap_values, label):
    # 시각화 함수
    fig, ax = plt.subplots(1, 2, figsize=(10, 5))

    # 원본 이미지
    ax[0].imshow(image)
    ax[0].set_title(f"원본 이미지({label})")

    # SHAP 시각화
    shap_image = np.sum(shap_values, axis=0) # 채널별 SHAP 값 합산
    ax[1].imshow(image, alpha=0.6) # 원본 이미지 투명하게 배경으로 표시
    ax[1].imshow(shap_image, cmap='coolwarm', alpha=0.5)
    ax[1].set_title(f"SHAP Explanation ({label})")

    plt.show()

In [ ]:
# 분석 실행
image_path = r"C:\Users\USER\Desktop\my_git\safebaby-xai\resnet50_explain\resnet50_sample_data\V5\back\b0010_bright_sr.png"

#단일 이미지 분석 실행
print(f"분석 중: {image_path}")
analyze_single_image(image_path)  # 단일 이미지 경로를 직접 전달

분석 중: C:\Users\USER\Desktop\my_git\safebaby-xai\resnet50_explain\resnet50_sample_data\V5\back\b0010_bright_sr.png
 예측 클래스: Back (확률: 97.41%)


RuntimeError: Output 0 of BackwardHookFunctionBackward is a view and is being modified inplace. This view was created inside a custom Function (or because an input was returned as-is) and the autograd logic to handle view+inplace would override the custom backward associated with the custom Function, leading to incorrect gradients. This behavior is forbidden. You can fix this by cloning the output of the custom Function.